In [ ]:
# 1. Import libraries
import opendatasets as od
import pandas as pd
import numpy as np
import os

# 2. Download dataset (you'll need your Kaggle credentials)
dataset_url = 'https://www.kaggle.com/datasets/godfredaddaiamoako/trotro'
od.download_kaggle(dataset_url)

# 3. Load the data
data_dir = 'trotro'  # or the directory where data was downloaded
data_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# 4. Basic data cleaning
def clean_data(df):
    # Handle missing values
    df = df.dropna(how='all')  # Drop completely empty rows
    df = df.fillna('Unknown')  # Fill other missing values
    
    # Convert data types
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str)
    
    return df

# 5. Process each file
for file in data_files:
    file_path = os.path.join(data_dir, file)
    df = pd.read_csv(file_path)
    df_clean = clean_data(df)
    df_clean.to_csv(f'cleaned_{file}', index=False)


# Basic analysis
for file in data_files:
    df = pd.read_csv(f'cleaned_{file}')
    print(f"\n--- {file} ---")
    print("Shape:", df.shape)
    print("\nMissing values:")
    print(df.isnull().sum())
    print("\nData types:")
    print(df.dtypes)
    print("\nSample data:")
    display(df.head())

# Example: Combine data from different files if needed
# This will depend on your specific dataset structure

# Save combined data
combined_data.to_csv('combined_trotro_data.csv', index=False)



: 